# Excercises 
# 1. Tune the network
Run the experiment below, explore the different parameters (see suggestions below) and study the result with tensorboard. 
Make a single page (1 a4) report of your findings. Use your visualisation skills to communicate your most important findings.

In [1]:
from mads_datasets import DatasetFactoryProvider, DatasetType

from mltrainer.preprocessors import BasePreprocessor
from mltrainer import imagemodels, Trainer, TrainerSettings, ReportTypes, metrics

import torch.optim as optim
from torch import nn
from tomlserializer import TOMLSerializer

In [2]:
import torch

print("torch version:", torch.__version__)
print("torch cuda version:", torch.version.cuda)
print("cuda available:", torch.cuda.is_available())
print("cuda device count:", torch.cuda.device_count())

torch version: 2.8.0+cpu
torch cuda version: None
cuda available: False
cuda device count: 0


In [3]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

using cpu


We will be using `tomlserializer` to easily keep track of our experiments, and to easily save the different things we did during our experiments.
It can export things like settings and models to a simple `toml` file, which can be easily shared, checked and modified.

First, we need the data. 

In [4]:
fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
preprocessor = BasePreprocessor()
streamers = fashionfactory.create_datastreamer(batchsize=64, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

2025-09-21 14:34:18.808 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\mwien\.cache\mads_datasets\fashionmnist
2025-09-21 14:34:18.810 | INFO     | mads_datasets.base:download_data:124 - File already exists at C:\Users\mwien\.cache\mads_datasets\fashionmnist\fashionmnist.pt
2025-09-21 14:34:18.810 | INFO     | mads_datasets.base:download_data:124 - File already exists at C:\Users\mwien\.cache\mads_datasets\fashionmnist\fashionmnist.pt


We need a way to determine how well our model is performing. We will use accuracy as a metric.

In [5]:
accuracy = metrics.Accuracy()

You can set up a single experiment.

- We will show the model batches of 64 images, 
- and for every epoch we will show the model 100 batches (trainsteps=100).
- then, we will test how well the model is doing on unseen data (teststeps=100).
- we will report our results during training to tensorboard, and report all configuration to a toml file.
- we will log the results into a directory called "modellogs", but you could change this to whatever you want.

In [6]:
import torch
loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=3,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps=100,
    valid_steps=100,
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
)


We will use a very basic model: a model with three linear layers.

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.units1 = units1
        self.units2 = units2
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, units1),
            nn.ReLU(),
            nn.Linear(units1, units2),
            nn.ReLU(),
            nn.Linear(units2, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork(
    num_classes=10, units1=256, units2=256)

I developped the `tomlserializer` package, it is a useful tool to save configs, models and settings as a tomlfile; that way it is easy to track what you changed during your experiments.

This package will 1. check if there is a `__dict__` attribute available, and if so, it will use that to extract the parameters that do not start with an underscore, like this:

In [8]:
{k: v for k, v in model.__dict__.items() if not k.startswith("_")}

{'training': True, 'num_classes': 10, 'units1': 256, 'units2': 256}

This means that if you want to add more parameters to the `.toml` file, eg `units3`, you can add them to the class like this:

```python
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int, units3: int) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.units1 = units1
        self.units2 = units2
        self.units3 = units3  # <-- add this line
```

And then it will be added to the `.toml` file. Check the result for yourself by using the `.save()` method of the `TomlSerializer` class like this:

In [9]:
tomlserializer = TOMLSerializer()
tomlserializer.save(settings, "settings.toml")
tomlserializer.save(model, "model.toml")

Check the `settings.toml` and `model.toml` files to see what is in there.

## Script for looping through some epochs

In [11]:
import torch

units = [64, 32, 16]
loss_fn = torch.nn.CrossEntropyLoss()

main_folder = "modellogs"
subfolder = "change_epochs"
amount_of_epochs = [5, 8, 10]

for epochs in amount_of_epochs:
    epoch_subfolder = f"{subfolder}/epochs_{epochs}"
    settings = TrainerSettings(
        epochs=epochs,
        metrics=[accuracy],
        logdir=f"{main_folder}/{epoch_subfolder}",
        train_steps=len(train),
        valid_steps=len(valid),
        reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
    )

    for unit1 in units:
        for unit2 in units:
            if unit2 <= unit1:
                print(f"Epochs: {epochs}, Units: {unit1}, {unit2}")
                model = NeuralNetwork(num_classes=10, units1=unit1, units2=unit2)

                trainer = Trainer(
                    model=model,
                    settings=settings,
                    loss_fn=loss_fn,
                    optimizer=optim.Adam,
                    traindataloader=trainstreamer,
                    validdataloader=validstreamer,
                    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
                )
                trainer.loop()

2025-09-21 14:36:51.313 | INFO     | mltrainer.settings:check_path:60 - Created logdir c:\Master Applied Data Science\Year 2\Semester 3 (Deep Learning & Model Deployment)\Portfolio-Marcello-Wienhoven\1-hypertuning-gridsearch\modellogs\change_epochs\epochs_5
2025-09-21 14:36:51.315 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs\change_epochs\epochs_5\20250921-143651
2025-09-21 14:36:51.316 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
2025-09-21 14:36:51.315 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs\change_epochs\epochs_5\20250921-143651
2025-09-21 14:36:51.316 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.


Epochs: 5, Units: 64, 64


100%|██████████| 937/937 [00:02<00:00, 390.34it/s]

2025-09-21 14:36:53.925 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 0.5916 test 0.4596 metric ['0.8371']
100%|██████████| 937/937 [00:02<00:00, 381.33it/s]

2025-09-21 14:36:56.644 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.4127 test 0.4087 metric ['0.8556']
100%|██████████| 937/937 [00:02<00:00, 354.05it/s]

2025-09-21 14:36:59.545 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.3672 test 0.3952 metric ['0.8587']
100%|██████████| 937/937 [00:02<00:00, 351.54it/s]

2025-09-21 14:37:02.477 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.3454 test 0.3744 metric ['0.8662']
100%|██████████| 937/937 [00:02<00:00, 331.55it/s]

2025-09-21 14:37:05.588 | INFO     | mltrainer.trainer:report:209 - Epoch 4 train 0.3252 test 0.3644 metric ['0.8706']
100%|██████████| 5/5 [00:14<00:00,  2.85s/it]
2025-09-21 14:37:05.590 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs\

Epochs: 5, Units: 64, 32


100%|██████████| 937/937 [00:02<00:00, 403.57it/s]

2025-09-21 14:37:08.122 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 0.6152 test 0.4715 metric ['0.8331']
100%|██████████| 937/937 [00:02<00:00, 379.21it/s]

2025-09-21 14:37:10.861 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.4289 test 0.4586 metric ['0.8352']
100%|██████████| 937/937 [00:02<00:00, 332.24it/s]

2025-09-21 14:37:13.947 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.3887 test 0.4082 metric ['0.8557']
100%|██████████| 937/937 [00:02<00:00, 342.66it/s]

2025-09-21 14:37:16.955 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.3640 test 0.3838 metric ['0.8620']
100%|██████████| 937/937 [00:02<00:00, 323.55it/s]

2025-09-21 14:37:20.136 | INFO     | mltrainer.trainer:report:209 - Epoch 4 train 0.3432 test 0.3841 metric ['0.8652']
2025-09-21 14:37:20.137 | INFO     | mltrainer.trainer:__call__:252 - best loss: 0.3838, current loss 0.3841.Counter 1/10.
100%|██████████| 5/5 [

...

100%|██████████| 937/937 [00:02<00:00, 453.76it/s]

2025-09-21 14:42:14.099 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 0.6376 test 0.4816 metric ['0.8325']
100%|██████████| 937/937 [00:02<00:00, 455.15it/s]

2025-09-21 14:42:16.381 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.4357 test 0.4502 metric ['0.8406']
100%|██████████| 937/937 [00:02<00:00, 444.56it/s]

2025-09-21 14:42:18.710 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.3984 test 0.4089 metric ['0.8547']
100%|██████████| 937/937 [00:02<00:00, 434.68it/s]

2025-09-21 14:42:21.083 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.3747 test 0.3946 metric ['0.8602']
100%|██████████| 937/937 [00:02<00:00, 446.44it/s]

2025-09-21 14:42:23.405 | INFO     | mltrainer.trainer:report:209 - Epoch 4 train 0.3583 test 0.3889 metric ['0.8611']
100%|██████████| 937/937 [00:02<00:00, 397.01it/s]

2025-09-21 14:42:25.975 | INFO     | mltrainer.trainer:report:209 - Epoch 5 train 0.3421 test

Epochs: 10, Units: 32, 16


100%|██████████| 937/937 [00:02<00:00, 391.28it/s]

2025-09-21 14:42:38.762 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 0.6755 test 0.5010 metric ['0.8278']
100%|██████████| 937/937 [00:02<00:00, 358.49it/s]

2025-09-21 14:42:41.646 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.4551 test 0.4563 metric ['0.8367']
100%|██████████| 937/937 [00:02<00:00, 363.36it/s]

2025-09-21 14:42:44.486 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.4160 test 0.4334 metric ['0.8488']
100%|██████████| 937/937 [00:02<00:00, 375.35it/s]

2025-09-21 14:42:47.251 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.3924 test 0.4265 metric ['0.8513']
100%|██████████| 937/937 [00:02<00:00, 375.63it/s]

2025-09-21 14:42:50.005 | INFO     | mltrainer.trainer:report:209 - Epoch 4 train 0.3760 test 0.4150 metric ['0.8519']
100%|██████████| 937/937 [00:02<00:00, 367.02it/s]

2025-09-21 14:42:52.830 | INFO     | mltrainer.trainer:report:209 - Epoch 5 train 0.3639 test

Epochs: 10, Units: 16, 16


100%|██████████| 937/937 [00:02<00:00, 344.78it/s]

2025-09-21 14:43:08.415 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 0.7487 test 0.5465 metric ['0.8170']
100%|██████████| 937/937 [00:02<00:00, 371.01it/s]

2025-09-21 14:43:11.192 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.4829 test 0.4854 metric ['0.8285']
100%|██████████| 937/937 [00:02<00:00, 349.68it/s]

2025-09-21 14:43:14.106 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.4423 test 0.4601 metric ['0.8403']
100%|██████████| 937/937 [00:02<00:00, 437.34it/s]

2025-09-21 14:43:16.526 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.4190 test 0.4486 metric ['0.8431']
100%|██████████| 937/937 [00:03<00:00, 308.74it/s]

2025-09-21 14:43:19.785 | INFO     | mltrainer.trainer:report:209 - Epoch 4 train 0.4047 test 0.4303 metric ['0.8515']
100%|██████████| 937/937 [00:02<00:00, 446.78it/s]

2025-09-21 14:43:22.116 | INFO     | mltrainer.trainer:report:209 - Epoch 5 train 0.3908 test

## Script for changing units

In [ ]:
import torch

units = [2**i for i in range(4, 11)]  # 16, 32, 64, ..., 1024
loss_fn = torch.nn.CrossEntropyLoss()

main_folder = "modellogs"
subfolder = "change_units"

for unit1 in units:
    for unit2 in units:
        if unit2 <= unit1:
            run_subfolder = f"{subfolder}/units_{unit1}_{unit2}"
            settings = TrainerSettings(
                epochs=3,
                metrics=[accuracy],
                logdir=f"{main_folder}/{run_subfolder}",
                train_steps=len(train),
                valid_steps=len(valid),
                reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
            )
            print(f"Units: {unit1}, {unit2}")
            model = NeuralNetwork(num_classes=10, units1=unit1, units2=unit2)

            trainer = Trainer(
                model=model,
                settings=settings,
                loss_fn=loss_fn,
                optimizer=optim.Adam,
                traindataloader=trainstreamer,
                validdataloader=validstreamer,
                scheduler=optim.lr_scheduler.ReduceLROnPlateau,
            )
            trainer.loop()

## Script for changing batchsize

In [ ]:
import torch

units = [64, 32, 16]
loss_fn = torch.nn.CrossEntropyLoss()

main_folder = "modellogs"
subfolder = "change_units"

settings = TrainerSettings(
    epochs=3,
    metrics=[accuracy],
    logdir=f"{main_folder}/{subfolder}",
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
)

for unit1 in units:
    for unit2 in units:
        if unit2 <= unit1:
            print(f"Units: {unit1}, {unit2}")
            model = NeuralNetwork(num_classes=10, units1=unit1, units2=unit2)

            trainer = Trainer(
                model=model,
                settings=settings,
                loss_fn=loss_fn,
                optimizer=optim.Adam,
                traindataloader=trainstreamer,
                validdataloader=validstreamer,
                scheduler=optim.lr_scheduler.ReduceLROnPlateau
            )
            trainer.loop()


2025-09-13 18:32:40.492 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs\20250913-183240
2025-09-13 18:32:40.493 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 937/937 [00:02<00:00, 366.04it/s]
2025-09-13 18:32:43.278 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 0.5857 test 0.4655 metric ['0.8346']
100%|██████████| 937/937 [00:02<00:00, 334.28it/s]
2025-09-13 18:32:46.325 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.4117 test 0.4120 metric ['0.8553']
100%|██████████| 937/937 [00:02<00:00, 327.81it/s]
2025-09-13 18:32:49.414 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.3694 test 0.3965 metric ['0.8589']
100%|██████████| 3/3 [00:08<00:00,  2.97s/it]
2025-09-13 18:32:49.417 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs\20250913-183249
2025-09-13 18:32:49.418 | INFO     | mltrainer.trainer:__init__:68

Because we have set the ReportType to TOML, you will find in every log dir a model.toml and settings.toml file.

Run the experiment, and study the result with tensorboard. 

Locally, it is easy to do that with VS code itself. On the server, you have to take these steps:

- in the terminal, `cd` to the location of the repository
- activate the python environment for the shell. This can be done with `.venv\Scripts\activate`.
- run `tensorboard --logdir=1-hypertuning-gridsearch/modellogs` in the terminal
- tensorboard will launch at `localhost:6006` and vscode will notify you that the port is forwarded
- you can either press the `launch` button in VScode or open your local browser at `localhost:6006`